In [12]:
R = "Ra"
print(R[0] + chr(ord(R[1])+1))

Rb


In [2]:
from __future__ import division
from sympy import *

In [3]:
def get_scattering_matrix(X, v_n, _i, j_etc):
    '''
    params
        X: a square MNA matrix as described https://pureadmin.qub.ac.uk/ws/portalfiles/portal/158209014/1570255463.pdf after the datum node has been removed
            entries in X are expected to be in terms of conductances symbols (GA, GB, GC...)
        v_n: number of nodes through the adaptor
        i: number of resistive sources attached to the adaptor
        j_etc: number of VCVS, nullors, etc.
    '''
    I = eye(_i)
    R = eye(_i)
    str = "Ra"
    # create R matrix and R symbols
    for n in range(_i):
        R[n,n] = symbols(str)
        str = str[0] + chr(ord(str[1]) + 1)
    print("inverting X...")
    Xinv = X.inv()
    zRz = zeros(_i, v_n).row_join(R).row_join(zeros(_i, j_etc))
    zIzT = zeros(_i, v_n).row_join(I).row_join(zeros(_i, j_etc)).T
    print("computing scattering matrix S...")
    S = I + 2*zRz*Xinv*zIzT
    str = "GA"
    print("substituting resistances...")
    for n in range(_i):
        G = symbols(str)
        S = S.subs({G: 1/(R[n,n])})
        str = str[0] + chr(ord(str[1]) + 1)
    print("simplifying...")
    S = S.applyfunc(simplify)
    return S

In [4]:
GA = symbols('GA')
GB = symbols('GB')
GC = symbols('GC')
GD = symbols('GD')
GE = symbols('GE')
GF = symbols('GF')

# Circuits from here: https://1.bp.blogspot.com/-FFC5U2CNcKM/WcVUpTksN0I/AAAAAAAAAsY/U6cdl102b24YAOVcN5ppeBseXtux4QKxgCLcBGAs/s1600/Thermionic%2BDistortion%2BSchematic%2BHighlighted.png

'''
Bass control
A:
B:
C:
D:
E:
F:
'''
MNA_BassControl = Matrix([
    [GD+GE, 0, 0, 0, 0, 0, 0, -GD, -GE, 0, 0, -1, 0, 0, 0, 0, 1],
    [0, GB, 0, 0, 0, -GB, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0],
    [0, 0, GC+GF, 0, 0, 0, -GC, 0, 0, -GF, 0, 0, 0, -1, 0, 0, 0],
    [0, 0, 0, GA, -GA, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, -GA, GA, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, -GB, 0, 0, 0, GB, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, -GC, 0, 0, 0, GC, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [-GD, 0, 0, 0, 0, 0, 0, GD, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [-GE, 0, 0, 0, 0, 0, 0, 0, GE, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, -GF, 0, 0, 0, 0, 0, 0, GF, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [-1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, -1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, -1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
])

'''
2nd gain stage
A: parallel adaptor connecting diode pair and GAIN pot
B: 47pF capacitor
C: load resistor
D: resistive voltage source (signal in)
E: series adaptor connecting 100nF capacitor and 22K resistor
F: 22K resistor
'''
MNA_GainStage2 = Matrix([
    [GA+GB+GC, 0, 0, 0, 0, 0, 0, -GA, -GB, -GC, 0, 0, 0, 0, 0, 0, 1],
    [0, GD, -GD, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, -GD, GD, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, GE, -GE, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, -GE, GE, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, -1, 0],
    [0, 0, 0, 0, 0, GF, -GF, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, -GF, GF, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0],
    [-GA, 0, 0, 0, 0, 0, 0, GA, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [-GB, 0, 0, 0, 0, 0, 0, 0, GB, 0, 0, 1, 0, 0, 0, 0, 0],
    [-GC, 0, 0, 0, 0, 0, 0, 0, 0, GC, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, -1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
])

'''
3rd gain stage
A: parallel adaptor connecting diode pair and 120pF capacitor
B: 220K resistor
C: 22K resistor
D: series adaptor connecting signal in, 47nF capacitor, and 22K resistor
E: resistive voltage source VREF
F: load resistor
'''
MNA_GainStage3 = Matrix([
    [GA+GC+GF, 0, 0, 0, 0, -GC, 0, 0, -GA, -GF, 0, 0, 0, 0, 0, 0, 1],
    [0, GE, -GE, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, -GE, GD+GE, -GD, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, -GD, GD, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 0, 0, 0, 0],
    [-GC, 0, 0, 0, 0, GC, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, GB, -GB, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, -GB, GB, 0, 0, -1, 0, 0, 0, 0, 0, 0],
    [-GA, 0, 0, 0, 0, 0, 0, 0, GA, 0, 1, 0, 0, 0, 0, 0, 0],
    [-GF, 0, 0, 0, 0, 0, 0, 0, 0, GF, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
])

'''
4th gain stage
A: ideal voltage source VREF
B: 10K resistive voltage source (signal in)
C: 10K resistor
D: parallel adaptor connecting 100pF capacitor and TRIM pot
E: load resistor
'''
MNA_GainStage4 = Matrix([
    [GD+GE, 0, 0, 0, 0, 0, 0, -GD, -GE, 0, 0, 0, 0, 0, 1],
    [0, GA, -GA, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, -GA, GA, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, GB, -GB, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, -GB, GB, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0],
    [0, 0, 0, 0, 0, GC, -GC, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, -GC, GC, 0, 0, 0, 0, 0, -1, 0, 0],
    [-GD, 0, 0, 0, 0, 0, 0, GD, 0, 0, 0, 0, 1, 0, 0],
    [-GE, 0, 0, 0, 0, 0, 0, 0, GE, 0, 0, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
])

In [18]:
S_BassControl = get_scattering_matrix(MNA_BassControl, 10, 6, 1)

inverting X...
computing scattering matrix S...
substituting resistances...
simplifying...


In [20]:
display(S_BassControl)

Matrix([
[                                                                        1,                                                                0,                                                               0,                                                               0,  0,                                                                0],
[                                 -2*Rb*Rd/(Rb*Rf + Rc*Rd + Rc*Rf + Rd*Rf), (-Rb*Rf + Rc*Rd + Rc*Rf + Rd*Rf)/(Rb*Rf + Rc*Rd + Rc*Rf + Rd*Rf),                 -2*Rb*(Rd + Rf)/(Rb*Rf + Rc*Rd + Rc*Rf + Rd*Rf),                        -2*Rb*Rf/(Rb*Rf + Rc*Rd + Rc*Rf + Rd*Rf),  0,                          2*Rb*Rd/(Rb*Rf + Rc*Rd + Rc*Rf + Rd*Rf)],
[                                 -2*Rc*Rd/(Rb*Rf + Rc*Rd + Rc*Rf + Rd*Rf),                         -2*Rc*Rf/(Rb*Rf + Rc*Rd + Rc*Rf + Rd*Rf), (Rb*Rf - Rc*Rd - Rc*Rf + Rd*Rf)/(Rb*Rf + Rc*Rd + Rc*Rf + Rd*Rf),                        -2*Rc*Rf/(Rb*Rf + Rc*Rd + Rc*Rf + Rd*Rf),  0,                          2*Rc

In [5]:
S_GainStage2 = get_scattering_matrix(MNA_GainStage2, 10, 6, 1)
display(S_GainStage2)

inverting X...
computing scattering matrix S...
substituting resistances...
simplifying...


Matrix([
[(-Ra + Rb + Rf)/(Ra + Rb + Rf),           2*Ra/(Ra + Rb + Rf),  0,                                   2*Ra*Rb/(Re*(Ra + Rb + Rf)),        -2*Ra*Rb/(Re*(Ra + Rb + Rf)),          -2*Ra/(Ra + Rb + Rf)],
[           2*Rb/(Ra + Rb + Rf), (Ra - Rb + Rf)/(Ra + Rb + Rf),  0,                            2*Rb*(Ra + Rf)/(Re*(Ra + Rb + Rf)), -2*Rb*(Ra + Rf)/(Re*(Ra + Rb + Rf)),           2*Rb/(Ra + Rb + Rf)],
[           2*Rb/(Ra + Rb + Rf),    2*(Ra + Rf)/(Ra + Rb + Rf), -1, 2*(Ra*Rb + Ra*Re + Rb*Re + Rb*Rf + Re*Rf)/(Re*(Ra + Rb + Rf)), -2*Rb*(Ra + Rf)/(Re*(Ra + Rb + Rf)),           2*Rb/(Ra + Rb + Rf)],
[                             0,                             0,  0,                                                             1,                                   0,                             0],
[                             0,                             0,  0,                                                             2,                                  -1,                        

In [6]:
S_GainStage3 = get_scattering_matrix(MNA_GainStage3, 10, 6, 1)
display(S_GainStage3)

inverting X...
computing scattering matrix S...
substituting resistances...
simplifying...


Matrix([
[(-Ra + Rb + Rc)/(Ra + Rb + Rc),          -2*Ra/(Ra + Rb + Rc),           2*Ra/(Ra + Rb + Rc),                    0,                   0,  0],
[          -2*Rb/(Ra + Rb + Rc), (Ra - Rb + Rc)/(Ra + Rb + Rc),           2*Rb/(Ra + Rb + Rc),                    0,                   0,  0],
[           2*Rc/(Ra + Rb + Rc),           2*Rc/(Ra + Rb + Rc), (Ra + Rb - Rc)/(Ra + Rb + Rc),                    0,                   0,  0],
[                             0,                             0,                             0, (-Rd + Re)/(Rd + Re),      2*Rd/(Rd + Re),  0],
[                             0,                             0,                             0,       2*Re/(Rd + Re), (Rd - Re)/(Rd + Re),  0],
[           2*Rc/(Ra + Rb + Rc),           2*Rc/(Ra + Rb + Rc),    2*(Ra + Rb)/(Ra + Rb + Rc),       2*Re/(Rd + Re),      2*Rd/(Rd + Re), -1]])

In [7]:
S_GainStage4 = get_scattering_matrix(MNA_GainStage4, 9, 5, 1)
display(S_GainStage4)

inverting X...
computing scattering matrix S...
substituting resistances...
simplifying...


Matrix([
[                  1,                 0, 0, 0,  0],
[                  2,                -1, 0, 0,  0],
[            2*Rc/Rb,          -2*Rc/Rb, 1, 0,  0],
[            2*Rd/Rb,          -2*Rd/Rb, 0, 1,  0],
[2*(Rb + Rc + Rd)/Rb, -(2*Rc + 2*Rd)/Rb, 2, 2, -1]])

For gain stages 2 and 3, port A of the R-type adaptor can successfully be adapted by setting Ra such that S<sub>aa</sub> = 0.

| Gain Stage | Ra |
| ---------- | -- |
| 2 | Rb + Rf |
| 3 | Rb + Rc | 

In [10]:
Ra = symbols("Ra")
Rb = symbols("Rb")
Rc = symbols("Rc")
Rf = symbols("Rf")
S_GainStage2 = S_GainStage2.subs({Ra: Rb+Rf}).applyfunc(simplify)
S_GainStage3 = S_GainStage3.subs({Ra: Rb+Rc}).applyfunc(simplify)
print("Gain Stage 2 scattering matrix, port A adapted:")
display(S_GainStage2)
print("Gain Stage 3 scattering matrix, port A adapted:")
display(S_GainStage3)

Gain Stage 2 scattering matrix, port A adapted:


Matrix([
[            0,                     1,  0,                                                Rb/Re,                         -Rb/Re,           -1],
[ Rb/(Rb + Rf),          Rf/(Rb + Rf),  0,                        Rb*(Rb + 2*Rf)/(Re*(Rb + Rf)), -Rb*(Rb + 2*Rf)/(Re*(Rb + Rf)), Rb/(Rb + Rf)],
[ Rb/(Rb + Rf), (Rb + 2*Rf)/(Rb + Rf), -1, (Rb**2 + 2*Rb*Re + 2*Rb*Rf + 2*Re*Rf)/(Re*(Rb + Rf)), -Rb*(Rb + 2*Rf)/(Re*(Rb + Rf)), Rb/(Rb + Rf)],
[            0,                     0,  0,                                                    1,                              0,            0],
[            0,                     0,  0,                                                    2,                             -1,            0],
[-Rf/(Rb + Rf),          Rf/(Rb + Rf),  0,                                 Rb*Rf/(Re*(Rb + Rf)),          -Rb*Rf/(Re*(Rb + Rf)), Rb/(Rb + Rf)]])

Gain Stage 3 scattering matrix, port A adapted:


Matrix([
[            0,           -1,                     1,                    0,                   0,  0],
[-Rb/(Rb + Rc), Rc/(Rb + Rc),          Rb/(Rb + Rc),                    0,                   0,  0],
[ Rc/(Rb + Rc), Rc/(Rb + Rc),          Rb/(Rb + Rc),                    0,                   0,  0],
[            0,            0,                     0, (-Rd + Re)/(Rd + Re),      2*Rd/(Rd + Re),  0],
[            0,            0,                     0,       2*Re/(Rd + Re), (Rd - Re)/(Rd + Re),  0],
[ Rc/(Rb + Rc), Rc/(Rb + Rc), (2*Rb + Rc)/(Rb + Rc),       2*Re/(Rd + Re),      2*Rd/(Rd + Re), -1]])

#Gain Stage 2 scattering matrix, port A adapted:
Matrix([
[            0,                     1,  0,                                                Rb/Re,                         -Rb/Re,           -1],
[ Rb/(Rb + Rf),          Rf/(Rb + Rf),  0,                        Rb*(Rb + 2*Rf)/(Re*(Rb + Rf)), -Rb*(Rb + 2*Rf)/(Re*(Rb + Rf)), Rb/(Rb + Rf)],
[ Rb/(Rb + Rf), (Rb + 2*Rf)/(Rb + Rf), -1, (Rb**2 + 2*Rb*Re + 2*Rb*Rf + 2*Re*Rf)/(Re*(Rb + Rf)), -Rb*(Rb + 2*Rf)/(Re*(Rb + Rf)), Rb/(Rb + Rf)],
[            0,                     0,  0,                                                    1,                              0,            0],
[            0,                     0,  0,                                                    2,                             -1,            0],
[-Rf/(Rb + Rf),          Rf/(Rb + Rf),  0,                                 Rb*Rf/(Re*(Rb + Rf)),          -Rb*Rf/(Re*(Rb + Rf)), Rb/(Rb + Rf)]])

#Gain Stage 3 scattering matrix, port A adapted:
Matrix([
[            0,           -1,                     1,                    0,                   0,  0],
[-Rb/(Rb + Rc), Rc/(Rb + Rc),          Rb/(Rb + Rc),                    0,                   0,  0],
[ Rc/(Rb + Rc), Rc/(Rb + Rc),          Rb/(Rb + Rc),                    0,                   0,  0],
[            0,            0,                     0, (-Rd + Re)/(Rd + Re),      2*Rd/(Rd + Re),  0],
[            0,            0,                     0,       2*Re/(Rd + Re), (Rd - Re)/(Rd + Re),  0],
[ Rc/(Rb + Rc), Rc/(Rb + Rc), (2*Rb + Rc)/(Rb + Rc),       2*Re/(Rd + Re),      2*Rd/(Rd + Re), -1]])